In [17]:
import sys
sys.path.append("..")

In [18]:
import plap
from plap.parameterization.parameterizer import Parameterizer

preprocessor = plap.Preprocessor(window_type="hamming")
parameterizer = Parameterizer(audio_path="../tests/data/redhot.wav", preprocessor=preprocessor)

parameterizer.signal.shape, parameterizer.sample_rate

INIT MPEG7: Unhandled case


((661500,), 22050)

# Parameterize Speech DB

In [23]:
import os
import numpy as np
import plap

# Define the paths to the subdirectories
male_folder = './audios/Speech/Male'
female_folder = './audios/Speech/Female'

# Initialize a list to hold the file names
all_files = []

# Get the list of all files in the Male folder and collect filenames
for file_name in os.listdir(male_folder):
    file_path = os.path.join(male_folder, file_name)
    if os.path.isfile(file_path):  # Check if it's a file
        all_files.append(file_path)

# Get the list of all files in the Female folder and collect filenames
for file_name in os.listdir(female_folder):
    file_path = os.path.join(female_folder, file_name)
    if os.path.isfile(file_path):  # Check if it's a file
        all_files.append(file_path)

# Sort the list of files alphabetically
all_files = sorted(all_files)

# Display the collected file names
# print('List of all files in Male and Female folders:')
# for file in all_files:
#     print(file)


# Compare implementations

In [9]:
import pandas as pd

def csv_values_to_numpy(file_path) -> np.ndarray:
    # Read the file and split values by commas
    array = np.loadtxt(file_path, delimiter=',')
    # Convert the data into a numpy array of floats
    numpy_array = np.array(array, dtype=float)
    return numpy_array

def compare_two_arrays(array1: np.ndarray, array2: np.ndarray, features: list):
    diff = np.abs(array1-array2)

    # Calculate mean, min, max, and std for each feature and store them
    mean_diffs = []
    min_diffs = []
    max_diffs = []
    mse_diffs = []
    offset = 0
    for feature in features:
        num_values = feature[1]
        mean_diffs.append(np.mean(diff[:, offset:offset+num_values]))
        min_diffs.append(np.min(diff[:, offset:offset+num_values]))
        max_diffs.append(np.max(diff[:, offset:offset+num_values]))
        mse_diffs.append(np.mean(diff[:, offset:offset+num_values]**2))
        offset += num_values
    
    # Create a dictionary with the collected data
    data = {
        'Mean difference': mean_diffs,
        'Min difference': min_diffs,
        'Max difference': max_diffs,
        'Mean Squared Error': mse_diffs,
    }

    # Create a DataFrame with the statistics
    feature_names = [f[0] for f in features]
    statistics_df = pd.DataFrame(data, index=feature_names).T
    return statistics_df

    # DIRECT COMPARISON FOR VERIFICATION
    # combined = np.column_stack((array1, array2, diff))
    # df = pd.DataFrame(combined)
    # print(df)
    # print(df.to_string())


# Basic Spectral + Timbral

In [ ]:
# Use plap for parameterization
fvector = plap.FeatureVector("sc","lat","tc","aff","asc","ass","asf","hsc","hsd","hss","hsv")

# one_fifth_files = all_files[:len(all_files)//5]
one_tenth_files = all_files[:len(all_files)//10]
all_fvector = np.zeros((len(one_tenth_files), len(fvector.features)))
all_fvector.shape

i = 0
for file in one_tenth_files:
    print(f"Processing file: {i+1}")
    plap.parameterize(
        audio_path=file,
        fvector=fvector,
        preprocessor=preprocessor
    )
    all_fvector[i,:] = fvector.values
    i += 1

all_fvector

output_file = './python_csv/basic_spectral_plus_timbral.csv'
np.savetxt(output_file, all_fvector, delimiter=',')

In [21]:
matlab_fv_path = './matlab_csv/basic_spectral_plus_timbral.csv'
matlab_feature_vector = csv_values_to_numpy(matlab_fv_path)
matlab_feature_vector = matlab_feature_vector[:27]

# all_timbral_spectral_python = all_fvector
all_timbral_spectral_python = csv_values_to_numpy("./python_csv/basic_spectral_plus_timbral.csv")
matlab_feature_vector.shape, all_timbral_spectral_python.shape

((27, 11), (27, 11))

In [22]:
features_list = [('sc', 1), ('lat', 1), ('tc', 1), ('aff', 1), ('asc', 1), ('ass', 1), ('asf', 1), ('hsc', 1), ('hsd', 1), ('hss', 1), ('hsv', 1)]
stats = compare_two_arrays(matlab_feature_vector, all_timbral_spectral_python, features_list)
stats

,sc,lat,tc,aff,asc,ass,asf,hsc,hsd,hss,hsv
Mean difference,1.114863e+03,2.739845e-01,0.049222,32.569818,0.877171,0.059700,0.414025,1.158170e+03,0.012853,0.040122,0.046265
Min difference,5.796846e+02,3.885781e-16,0.002275,2.960468,0.666873,0.002969,0.359621,8.216762e+02,0.000456,0.000257,0.002605
Max difference,1.552563e+03,1.480764e+00,0.115678,119.155782,1.462979,0.198913,0.497601,1.607814e+03,0.038909,0.155493,0.087057
Mean Squared Error,1.299923e+06,2.321671e-01,0.003468,2123.359250,0.793616,0.005646,0.172354,1.374247e+06,0.000240,0.002850,0.002648


In [ ]:
# mean_sc_val = np.mean(all_timbral_spectral_python[:,0])
# mean_hsc_val = np.mean(all_timbral_spectral_python[:,1])
# mean_hsd_val = np.mean(all_timbral_spectral_python[:,2])
# mean_hss_val = np.mean(all_timbral_spectral_python[:,3])
# mean_hsv_val = np.mean(all_timbral_spectral_python[:,4])
# mean_hsc_val

for i in range(all_timbral_spectral_python.shape[1]):
    print(f"mean diff % of mean val of {features_list[i][0]}: {stats.iat[0,i] * 100 / np.mean(all_timbral_spectral_python[:,i])}")

percentile sc: 
percentile lat: 
percentile tc: 
percentile aff: 
percentile asc: 
percentile ass: 
percentile asf: 
percentile hsc: 
percentile hsd: 
percentile hss: 
percentile hsv: 


In [ ]:
percentile_sc = stats.iat[0,0] * 100 / mean_sc_val
percentile_hsc = stats.iat[0,1] * 100 / mean_hsc_val
percentile_hsd = stats.iat[0,2] * 100 / mean_hsd_val
percentile_hss = stats.iat[0,3] * 100 / mean_hss_val
percentile_hsv = stats.iat[0,4] * 100 / mean_hsv_val
[percentile_sc, percentile_hsc, percentile_hsd, percentile_hss, percentile_hsv]